In [1]:
!pip install  duckdb --upgrade
!pip install  deltalake --upgrade
!pip install  polars --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 37.0 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.9.2
    Uninstalling duckdb-0.9.2:
      Successfully uninstalled duckdb-0.9.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 10.5 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.2
    Uninstalling polars-0.20.2:
      Successfully uninstalled polars-0.20.2


In [2]:
from psutil import *
core = cpu_count()
vCPU = str(core) + " vCPU"
mem=round(virtual_memory().total/(1024 * 1024 * 1024),0)
print(vCPU +' Memory:'+ str(mem))

2 vCPU Memory:13.0


In [3]:
raw_landing='/lakehouse/default/Files/raw'

# Download Data

In [4]:
import re ,os
import requests
from urllib.request import urlopen
def download(url,Path):
    if not os.path.exists(Path):
      os.makedirs(Path, exist_ok=True)
    result = urlopen(url).read().decode('utf-8')
    pattern = re.compile(r'[\w.]*.zip')
    filelist1 = pattern.findall(result)
    filelist_unique = dict.fromkeys(filelist1)
    print(str(len(filelist_unique)) + ' New File Loaded')
    if len(filelist_unique) != 0 :
      for x in filelist_unique:
           with requests.get(url+x, stream=True) as resp:
            if resp.ok:
              with open(f"{Path}/{x}", "wb") as f:
               for chunk in resp.iter_content(chunk_size=4096):
                f.write(chunk)
    return "done"

In [5]:
download("https://nemweb.com.au/Reports/Current/Daily_Reports/",raw_landing+"/zip/")

60 New File Loaded


'done'

_**UNZIP**_

In [6]:
from shutil import unpack_archive
import os , glob
def unzip(Source, Destination):
    filelist=[os.path.basename(x) for x in glob.glob(Source+'*.zip')]
    for x in filelist:
          try:
            unpack_archive(str(Source+x), str(Destination), 'zip')
          except:
            pass
    return "done"

In [7]:
unzip(raw_landing+"/zip/",raw_landing+"/csv/")

'done'

# Data Transformation

_**DuckDB**_

In [8]:
%%time
from deltalake.writer import write_deltalake
import duckdb
raw =duckdb.sql(f"""from read_csv('{raw_landing}/csv/*.CSV',
    Skip=1,header =0,
    all_varchar=1,
    columns={{
    'I': 'VARCHAR','UNIT': 'VARCHAR','XX': 'VARCHAR','VERSION': 'VARCHAR','SETTLEMENTDATE': 'VARCHAR','RUNNO': 'VARCHAR',
    'DUID': 'VARCHAR','INTERVENTION': 'VARCHAR','DISPATCHMODE': 'VARCHAR','AGCSTATUS': 'VARCHAR','INITIALMW': 'VARCHAR',
    'TOTALCLEARED': 'VARCHAR','RAMPDOWNRATE': 'VARCHAR','RAMPUPRATE': 'VARCHAR','LOWER5MIN': 'VARCHAR',
    'LOWER60SEC': 'VARCHAR','LOWER6SEC': 'VARCHAR','RAISE5MIN': 'VARCHAR','RAISE60SEC': 'VARCHAR',
    'RAISE6SEC': 'VARCHAR','MARGINAL5MINVALUE': 'VARCHAR','MARGINAL60SECVALUE': 'VARCHAR',
    'MARGINAL6SECVALUE': 'VARCHAR','MARGINALVALUE': 'VARCHAR','VIOLATION5MINDEGREE': 'VARCHAR',
    'VIOLATION60SECDEGREE': 'VARCHAR','VIOLATION6SECDEGREE': 'VARCHAR','VIOLATIONDEGREE': 'VARCHAR',
    'LOWERREG': 'VARCHAR','RAISEREG': 'VARCHAR','AVAILABILITY': 'VARCHAR','RAISE6SECFLAGS': 'VARCHAR',
    'RAISE60SECFLAGS': 'VARCHAR','RAISE5MINFLAGS': 'VARCHAR','RAISEREGFLAGS': 'VARCHAR',
    'LOWER6SECFLAGS': 'VARCHAR','LOWER60SECFLAGS': 'VARCHAR','LOWER5MINFLAGS': 'VARCHAR',
    'LOWERREGFLAGS': 'VARCHAR','RAISEREGAVAILABILITY': 'VARCHAR','RAISEREGENABLEMENTMAX': 'VARCHAR',
    'RAISEREGENABLEMENTMIN': 'VARCHAR','LOWERREGAVAILABILITY': 'VARCHAR','LOWERREGENABLEMENTMAX': 'VARCHAR',
    'LOWERREGENABLEMENTMIN': 'VARCHAR','RAISE6SECACTUALAVAILABILITY': 'VARCHAR',
    'RAISE60SECACTUALAVAILABILITY': 'VARCHAR','RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
    'RAISEREGACTUALAVAILABILITY': 'VARCHAR','LOWER6SECACTUALAVAILABILITY': 'VARCHAR',
    'LOWER60SECACTUALAVAILABILITY': 'VARCHAR','LOWER5MINACTUALAVAILABILITY': 'VARCHAR','LOWERREGACTUALAVAILABILITY': 'VARCHAR'
    }},
    filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
    --- to extract the data you need to apply those filters
    where I='D' and UNIT ='DUNIT' AND VERSION = 3                  """)
columns = list(set(raw.columns) - {'SETTLEMENTDATE','DUID','I','filename','UNIT'})
exprs = [
      duckdb.ColumnExpression(x).cast(duckdb.typing.DOUBLE).alias(x)
      for x in columns
       ]
rel2 = raw.select('SETTLEMENTDATE','DUID','I','filename','UNIT',*exprs)
final_scada=duckdb.sql(""" select *exclude(SETTLEMENTDATE,I,XX,filename),
    cast (SETTLEMENTDATE as timestamp) as SETTLEMENTDATE,
    parse_filename(filename) as filename,
    isoyear (cast (SETTLEMENTDATE as timestamp)) as YEAR  from rel2  """)

write_deltalake("/lakehouse/default/Tables/scada_duckdb",final_scada.arrow(),
mode="overwrite",engine='rust',partition_by=['YEAR'],storage_options={"allow_unsafe_rename":"true"})

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 1min 48s, sys: 14.1 s, total: 2min 2s
Wall time: 1min 27s


_**Polars**_

In [9]:
%%time
#work in progress, I don't know how to parse datetime and read a csv with variable column numbers
import polars as pl
colmn=[
'I','UNIT','XX','VERSION','SETTLEMENTDATE','RUNNO',
    'DUID','INTERVENTION','DISPATCHMODE','AGCSTATUS','INITIALMW',
    'TOTALCLEARED','RAMPDOWNRATE','RAMPUPRATE','LOWER5MIN',
    'LOWER60SEC','LOWER6SEC','RAISE5MIN','RAISE60SEC',
    'RAISE6SEC','MARGINAL5MINVALUE','MARGINAL60SECVALUE',
    'MARGINAL6SECVALUE','MARGINALVALUE','VIOLATION5MINDEGREE'
]
raw = pl.scan_csv(f'{raw_landing}/csv/*.CSV',skip_rows=1, new_columns=colmn,has_header=False, infer_schema_length=0,truncate_ragged_lines=True)
transform =(
    raw
    .filter(pl.col("I")=='D')
    .filter(pl.col("UNIT")=='DUNIT')
    .filter(pl.col("VERSION")=='3')
    .drop("XX")
    .drop("I")
)
#transform=transform.with_columns(pl.col('SETTLEMENTDATE').str.strptime(pl.Date, 'yyyy/MM/dd HH:mm:ss').cast(pl.Datetime))
columns = list(set(transform.columns) - {'SETTLEMENTDATE','DUID','UNIT'})
for x in columns:
   transform=transform.with_columns(pl.col(x).cast(pl.Float64))
transform.collect(streaming=True)
write_deltalake("/lakehouse/default/Tables/scada_polars",transform.collect(streaming=True).to_arrow(),
mode="overwrite",engine='rust',storage_options={"allow_unsafe_rename":"true"})

CPU times: user 1min 35s, sys: 35.6 s, total: 2min 10s
Wall time: 2min 20s
